In [1]:
import pandas as pd
import numpy as np
import time

t = time.time()

dataAirtable = pd.read_csv("AT.csv", low_memory=False)
dataAirtable = dataAirtable.fillna(
    {
        'TR Product Name': '',
        'Product Name': '',
        'Origin': '',
        'Color Code': 'NA',
        'TR Description Package 1': '',
        'TR Description Package 2': '',
        'TR Description Package 3': '',
        'TR Description Package 4': '',
        'TR Description Package #5': '',
        'TR Description Package #6': '',
        'EN Description Package 1': '',
        'EN Description Package 2': '',
        'EN Description Package 3': '',
        'EN Description Package 4': '',
        'EN Description Package #5': '',
        'EN Description Package #6': '',
        'TR Composition Package 1': '',
        'TR Composition Package 2': '',
        'TR Composition Package 3': '',
        'TR Composition Package 4': '',
        'TR Composition Package 5': '',
        'TR Composition Package 6': '',
        'EN Composition Package 1': '',
        'EN Composition Package 2': '',
        'EN Composition Package 3': '',
        'EN Composition Package 4': '',
        'EN Composition Package 5': '',
        'EN Composition Package 6': '',
        'Compatibility': ''
    }
)
dataAirtable = dataAirtable.astype(
    {
        'TR Product Name': 'string',
        'Product Name': 'string',
        'Origin': 'string',
        'TR Description Package 1': 'string',
        'TR Description Package 2': 'string',
        'TR Description Package 3': 'string',
        'TR Description Package 4': 'string',
        'TR Description Package #5': 'string',
        'TR Description Package #6': 'string',
        'EN Description Package 1': 'string',
        'EN Description Package 2': 'string',
        'EN Description Package 3': 'string',
        'EN Description Package 4': 'string',
        'EN Description Package #5': 'string',
        'EN Description Package #6': 'string',
        'TR Composition Package 1': 'string',
        'TR Composition Package 2': 'string',
        'TR Composition Package 3': 'string',
        'TR Composition Package 4': 'string',
        'TR Composition Package 5': 'string',
        'TR Composition Package 6': 'string',
        'EN Composition Package 1': 'string',
        'EN Composition Package 2': 'string',
        'EN Composition Package 3': 'string',
        'EN Composition Package 4': 'string',
        'EN Composition Package 5': 'string',
        'EN Composition Package 6': 'string',
        'Compatibility': 'string'
    }
)

dataVendors = pd.read_excel('1_Vendors.xlsx')
dataVendors.fillna(
{
    'Sup. Code': '',
    'Supplier': '',
    'Src. Method': '',
    'VWH Code': '',
    'Lead Time': '',
    'MOQ': ''
})

dataProductGroups = pd.read_excel('2_ProductGroups.xlsx')
dataProductGroups.fillna(
{
    'FA Subcategory Code': ''
})

# Left join the "Product Groups" table to "Airtable" table
dataAirtable = dataAirtable.merge(dataProductGroups, how = 'left', on = 'Product Category')

# Left join the "Vendors" table to "Airtable" table
dataAirtable = dataAirtable.merge(dataVendors, how = 'left', on = 'Supplier')

# Define the BOM types
atBOMTypes = ["Single Item", "Assemblable", "Collection"]

marketDict = {
    "": [""],
    "All": ["ATH", "AUS", "BAR", "BSL", "BER", "BOS", "BRU", "CHI", "COP", "DEN", "DUB", "HKG", "IST", "LIS", "LON", "LA", "LUX", "MAD", "MIA", "MUC", "NY", "PAR", "SAN", "SF", "SEA", "SIN", "LON", "YVR", "VIE", "WDC", "ZUR"],
    "Athens": ["ATH"],
    "Austin": ["AUS"],
    "Barcelona": ["BAR"],
    "Basel": ["BSL"],
    "Berlin": ["BER"],
    "Boston": ["BOS"],
    "Brussels": ["BRU"],
    "Chicago": ["CHI"],
    "Copenhagen": ["COP"],
    "Core Markets": ["ATH", "IST", "DUB"],
    "Denver": ["DEN"],
    "Dubai": ["DUB"],
    "Hong Kong": ["HKG"],
    "Istanbul": ["IST"],
    "Lisbon": ["LIS"],
    "London": ["LON"],
    "Los Angeles": ["LA"],
    "Luxembourg": ["LUX"],
    "Madrid": ["MAD"],
    "Miami": ["MIA"],
    "Munich": ["MUC"],
    "New York": ["NY"],
    "Paris": ["PAR"],
    "San Diego": ["SAN"],
    "San Francisco": ["SF"],
    "Seattle": ["SEA"],
    "Singapore": ["SIN"],
    "UK": ["LON"],
    "USA": ["AUS", "BOS", "CHI", "DEN", "LA", "MIA", "NY","SAN", "SF", "SEA", "WDC"],
    "Vancouver": ["YVR"],
    "Vienna": ["VIE"],
    "Washington D.C.": ["WDC"],
    "Zurich": ["ZUR"],
    "TBD": ["BSL", "BRU", "MUC", "YVR"],
    "RDC": ["RDC01", "RDC02", "RDC03", "RDC04"],
    "VWH": ["ASYA", "MEX", "OYKU", "TUR", "USV"]
}

# Define Origin and Country dictionary
originDict = {
    "": "",
    "Austria": "AT",
    "Bulgaria": "BG",
    "Brazil": "BR",
    "Canada": "CA",
    "China": "CN",
    "Denmark": "DK",
    "France": "FR",
    "Germany": "DE",
    "Hong Kong": "HK",
    "India": "IN",
    "Lithuania": "LT",
    "Malaysia": "ML",
    "Mexico": "MX",
    "Mixed": "",
    "Poland": "PL",
    "Portugal": "PT",
    "Singapore": "SG",
    "Slovenia": "SI",
    "Sweden": "SE",
    "Turkey": "TR",
    "Ukraine": "UA",
    "Unknown": "",
    "United Kingdom": "GB",
    "United States": "US",
    "Vietnam": "VN"
}

# Define Item List array to be transformed to DataFrame
itemList = []
# Define Child List array to be transformed to DataFrame
BOMComponentList = []
# Define Compatibility List array to be transformed to DataFrame
compatibilityList = []

# Loop for "Airtable" data "Planning Item" rows
for index, row in dataAirtable.iterrows():
    # JSON-like mapping of multiple-column attributes
    atDescriptionPackage = [
        [row['TR Description Package 1'], row['TR Description Package 2'],row['TR Description Package 3'],row['TR Description Package 4'],row['TR Description Package #5'], row['TR Description Package #6']],
        [row['EN Description Package 1'], row['EN Description Package 2'],row['EN Description Package 3'],row['EN Description Package 4'],row['EN Description Package #5'], row['EN Description Package #6']]
    ]
    atCompositionPackage = [
        [row['TR Composition Package 1'], row['TR Composition Package 2'],row['TR Composition Package 3'],row['TR Composition Package 4'],row['TR Composition Package 5'], row['TR Composition Package 6']],
        [row['EN Composition Package 1'], row['EN Composition Package 2'],row['EN Composition Package 3'],row['EN Composition Package 4'],row['EN Composition Package 5'], row['EN Composition Package 6']]
    ]
    atDimensionPackage = [
        [row['Package #1 Length cm'], row['Package #1 Width cm'], row['Package #1 Height cm'], row['Package #1 Weight Kg']],
        [row['Package #2 Length cm'], row['Package #2 Width cm'], row['Package #2 Height cm'], row['Package #2 Weight Kg']],
        [row['Package #3 Length cm'], row['Package #3 Width cm'], row['Package #3 Height cm'], row['Package #3 Weight Kg']],
        [row['Package #4 Length cm'], row['Package #4 Width cm'], row['Package #4 Height cm'], row['Package #4 Weight Kg']],
        [row['Package #5 Length cm'], row['Package #5 Width cm'], row['Package #5 Height cm'], row['Package #5 Weight Kg']],
        [row['Package #6 Length cm'], row['Package #6 Width cm'], row['Package #6 Height cm'], row['Package #6 Weight Kg']]
    ]
    
    # Define the Parent EN Dscription fields
    atENDesc = row['Product Name']
    if (len(row['Product Name']) <= 50):
        atENDesc1 = row['Product Name']
        atENDesc2 = ''
    else:
        atENDesc1 = row['Product Name'][:row['Product Name'][:49].rindex(' ')]
        if (len(row['Product Name'][len(atENDesc1)+1:]) <= 50):
            atENDesc2 = row['Product Name'][len(atENDesc1)+1:]
        else:
            atENDesc2 = row['Product Name'][len(atENDesc1)+1:][:50]
    
    # Define the Parent TR Description fields
    atTRDesc = row['TR Product Name']
    if (len(row['TR Product Name']) <= 50):
        atTRDesc1 = row['TR Product Name']
        atTRDesc2 = ''
    else:
        atTRDesc1 = row['TR Product Name'][:row['TR Product Name'][:49].rindex(' ')]
        if (len(row['TR Product Name'][len(atTRDesc1)+1:]) <= 50):
            atTRDesc2 = row['TR Product Name'][len(atTRDesc1)+1:]
        else:
            atTRDesc2 = row['TR Product Name'][len(atTRDesc1)+1:][:50]

    # Define the Vendor Number
    atVendorNo = row['Vendor No.']
    
    # Define the Parent Package Dimension fields
    atWidth = 0
    atSize = 0
    atHeight = 0
    atGrossWeight = 0
    atVolume = 0
    if (row['# of parts'] == 1):
        atWidth = atDimensionPackage[0][1]
        atSize = atDimensionPackage[0][0]
        atHeight = atDimensionPackage[0][2]
        atGrossWeight = atDimensionPackage[0][3]*1000
    elif (row['# of parts'] <= 6):
        for i in range(int(row['# of parts'])):
            atGrossWeight += atDimensionPackage[i][3]*1000
            atVolume += atDimensionPackage[i][1]*atDimensionPackage[i][0]*atDimensionPackage[i][2]
        atWidth = round(atVolume**(1./3.),2)
        atSize = round(atVolume**(1./3.),2)
        atHeight = round(atVolume**(1./3.),2)
    else:
        atWidth = 0
        atSize = 0
        atHeight = 0
        atGrossWeight = 0
        atVolume = 0
    
    # Define the Parent Composition for Export fields
    tempTRComposition = []
    tempENComposition = []
    if (row['# of parts'] == 1):
        atTRComposition = atCompositionPackage[0][0]
        atENComposition = atCompositionPackage[1][0]
    elif (row['# of parts'] <= 6):
        for i in range(int(row['# of parts'])):
            tempTRComposition.append(atCompositionPackage[0][i])
            tempENComposition.append(atCompositionPackage[1][i])
        atTRComposition = ', '.join(tempTRComposition)
        atENComposition = ', '.join(tempENComposition)
    else:
        atTRComposition = ''
        atENComposition = ''
        
        
    # Define the Handmade field
    if (row['Handmade'] == 'Yes'):
        atHandmade = 'true'
    else:
        atHandmade = 'false'
    
    # Define the SC Hub fields
    if (row['Sourcing Method'] == 'Local Supplier'):
        atSCHubManaged = 'true'
        atSCHubSupplied = 'false'
    else:
        atSCHubManaged = 'true'
        atSCHubSupplied = 'true'
        
    # Define the Parent Billable field
    if (row['NAV Product Trees'] == 'Single Item' or row['NAV Product Trees'] == 'Assemblable'):
        atBillable = 'true'
    else:
        atBillable = 'false'
    
    # Define the BOM Type field
    if (row['NAV Product Trees'] == 'Single Item'):
        atBOMType = 'Single'
    elif (row['NAV Product Trees'] == 'Assemblable'):
        atBOMType = 'Assemblable'
    else:
        atBOMType = 'Collection'
    
    # Define the Parent WMS Code field
    if (row['NAV Product Trees'] == 'Single Item'):
        atWMSCode = row['Code']
    else:
        atWMSCode = ''
    
    # Define the Compatibility to Discrete City Values
    atCompatibilityList = []
    for compatibility in row['Compatibility'].split(','):
        for city in marketDict[compatibility]:
            atCompatibilityList.append(city)
    for rdc in marketDict['RDC']:
        atCompatibilityList.append(rdc)
    if (row['VWH Code'] is not np.nan):
        atCompatibilityList.append(row['VWH Code'])
    else:
        pass
    atCompatibilityList.sort()
        
    # Mapping of "Planning Item" attribute dictionary
    itemDict = {
        'No': row['Code'],
        'Description 1': atENDesc1 if (row['Sourcing Method'] == 'Local Supplier') else atTRDesc1,
        'Description 2': atENDesc2 if (row['Sourcing Method'] == 'Local Supplier') else atTRDesc2,
        'Base Unit of Measure': 'ADET',
        'Type': 'Inventory',
        'Inventory Posting Group': 'TİCARİ',
        'Vendor No.': atVendorNo,
        'Vendor Item No.': atDescriptionPackage[0][0] if (row['Sourcing Method'] == 'Local Supplier' and row['# of parts'] == 1) else '',
        'Lead Time Calculation': row['Lead Time'],
        'Gross Weight': atGrossWeight,
        'Tariff No.': '',
        'Gen. Prod. Posting Group': 'TİCARİ',
        'Country/Region of Origin Code': originDict[row['Origin']],
        'VAT Prod. Posting Group': 'KDVSIZ' if (row['Sourcing Method'] == 'Local Supplier') else '',
        'Minimum Order Quantity': row['MOQ'],
        'Sales Unit of Measure': 'ADET',
        'Purch. Unit of Measure': 'ADET',
        'Item Category Code': 'X',
        'Product Group Code': row['Product Group Code'],
        'Width (cm)': atWidth,
        'Size (cm)': atSize,
        'Height (cm)': atHeight,
        'EN Description 1': atENDesc1,
        'EN Description 2': atENDesc2,
        'TR Composition for Export': atTRComposition if (row['Sourcing Method'] == 'Global Supplier') else '',
        'EN Composition for Export': atENComposition if (row['Sourcing Method'] == 'Global Supplier') else '',
        'Supplier Name': row['Supplier'],
        'SKU Supplier Code': row['Supplier Code'],
        'SKU Item Code': row['Item Code'],
        'SKU Color Code': row['Color Code'],
        'Dimension Code': row['Dimension Code'],
        'New Contact': row['Contact'],
        'Product Length (cm)': row['Length (cm)'],
        'Product Width (cm)': row['Width (cm)'],
        'Product Height (cm)': row['Height (cm)'],
        'Handmade': atHandmade,
        'HS Code': '',
        'SC Hub Managed': atSCHubManaged,
        'SC Hub Supplied': atSCHubSupplied,
        'FDA Code': '',
        'Sent to WMS': 'false',
        'Billable': atBillable,
        'BOM Type': atBOMType,
        'WMS Code': atWMSCode,
        'ForSlim4': 'true',
        'Planning Item': 'true',
        'Stocked Item': 'true',
        'Logistic Item': 'true' if (row['# of parts'] == 1) else 'false',
        'VWH Code': row['VWH Code'],
        'Product Category': row['Product Category'],
        'Sourcing Method': row['Sourcing Method'],
        'Status': row['Status']
    }
    itemList.append(itemDict)
    
    # Mapping of "Item Compatibility" attribute dictionary
    for warehouse in atCompatibilityList:
        compatibilityDict = {
            'Item No.': row['Code'],
            'Parent Location': warehouse
        }
        compatibilityList.append(compatibilityDict)
        
    
    if (row['NAV Product Trees'] != 'Single Item' and row['# of parts'] != 1):
        for child in range(int(row['# of parts'])):
            
            atCodeChildTemp = row['Code'] + '-'+ str(child+1) + '/' + str(int(row['# of parts']))
            if (len(atCodeChildTemp) <= 20):
                atCodeChild = atCodeChildTemp
            else:
                atCodeChild = atCodeChildTemp[:atCodeChildTemp.find('-',4)]+atCodeChildTemp[atCodeChildTemp.find('-',4)+1:]
            
            # Define the Child TR Description fields
            if (row['# of parts'] <= 6):
                atTRDescChild = (row['TR Product Name'] + ' - ' + atDescriptionPackage[0][child])
                if (len(atTRDescChild) <= 50):
                    atTRDesc1Child = atTRDescChild
                    atTRDesc2Child = ''
                else:
                    atTRDesc1Child = atTRDescChild[:atTRDescChild[:49].rindex(' ')]
                    if (len(atTRDescChild[len(atTRDesc1Child)+1:]) <= 50):
                        atTRDesc2Child = atTRDescChild[len(atTRDesc1Child)+1:]
                    else:
                        atTRDesc2Child = atTRDescChild[len(atTRDesc1Child)+1:][:50]
            else:
                atTRDescChild = (row['TR Product Name'] + ' - ')
                if (len(atTRDescChild) <= 50):
                    atTRDesc1Child = atTRDescChild
                    atTRDesc2Child = ''
                else:
                    atTRDesc1Child = atTRDescChild[:atTRDescChild[:49].rindex(' ')]
                    if (len(atTRDescChild[len(atTRDesc1Child)+1:]) <= 50):
                        atTRDesc2Child = atTRDescChild[len(atTRDesc1Child)+1:]
                    else:
                        atTRDesc2Child = atTRDescChild[len(atTRDesc1Child)+1:][:50]
            
            if (row['# of parts'] <= 6):
                atVendorItemNoChild = atDescriptionPackage[0][child]
            else:
                atVendorItemNoChild = ''
            
            # Define the Child EN Description fields
            if (row['# of parts'] <= 6):
                atENDescChild = row['Product Name'] + ' - ' + atDescriptionPackage[1][child]
                if (len(atENDescChild) < 50):
                    atENDesc1Child = atENDescChild
                    atENDesc2Child = ''
                else:
                    atENDesc1Child = atENDescChild[:atENDescChild[:49].rindex(' ')]
                    if (len(atENDescChild[len(atENDesc1Child)+1:]) <= 50):
                        atENDesc2Child = atENDescChild[len(atENDesc1Child)+1:]
                    else:
                        atENDesc2Child = atENDescChild[len(atENDesc1Child)+1:][:50]
            else:
                atENDescChild = row['Product Name'] + ' - '
                if (len(atENDescChild) < 50):
                    atENDesc1Child = atENDescChild
                    atENDesc2Child = ''
                else:
                    atENDesc1Child = atENDescChild[:atENDescChild[:49].rindex(' ')]
                    if (len(atENDescChild[len(atENDesc1Child)+1:]) <= 50):
                        atENDesc2Child = atENDescChild[len(atENDesc1Child)+1:]
                    else:
                        atENDesc2Child = atENDescChild[len(atENDesc1Child)+1:][:50]
            
            # Define the Child Package Dimension fields
            if (row['# of parts'] <= 6):
                atWidthChild = atDimensionPackage[child][1]
                atSizeChild = atDimensionPackage[child][0]
                atHeightChild = atDimensionPackage[child][2]
                atGrossWeightChild = atDimensionPackage[child][3]*1000
            else:
                atWidthChild = 0
                atSizeChild = 0
                atHeightChild = 0
                atGrossWeightChild = 0
            
            # Define the Child Composition fields
            if (row['# of parts'] <= 6):
                atTRCompositionChild = atCompositionPackage[0][child]
                atENCompositionChild = atCompositionPackage[1][child]
            else:
                atTRCompositionChild = ''
                atENCompositionChild = ''
            
            # Define the Child Billable field
            atBillableChild = 'false'
            
            # Define the Child WMS Code field
            if (row['NAV Product Trees'] == 'Assemblable'):
                atWMSCodeChild = atCodeChild
            else:
                atWMSCodeChild = row['Code']
            
            # Define Line No:
            atLineNoChild = str(int((child+1)*10000))
            
            itemDictChild = {
                'No': atCodeChild,
                'Description 1': atENDesc1Child if (row['Sourcing Method'] == 'Local Supplier') else atTRDesc1Child,
                'Description 2': atENDesc2Child if (row['Sourcing Method'] == 'Local Supplier') else atTRDesc2Child,
                'Base Unit of Measure': 'ADET',
                'Type': 'Inventory',
                'Inventory Posting Group': 'TİCARİ',
                'Vendor No.': atVendorNo,
                'Vendor Item No.': atVendorItemNoChild if (row['Sourcing Method'] == 'Local Supplier') else '',
                'Lead Time Calculation': row['Lead Time'],
                'Gross Weight': atGrossWeightChild,
                'Tariff No.': '',
                'Gen. Prod. Posting Group': 'TİCARİ',
                'Country/Region of Origin Code': originDict[row['Origin']],
                'VAT Prod. Posting Group': 'KDVSIZ' if (row['Sourcing Method'] == 'Local Supplier') else '',
                'Minimum Order Quantity': row['MOQ'],
                'Sales Unit of Measure': 'ADET',
                'Purch. Unit of Measure': 'ADET',
                'Item Category Code': 'X',
                'Product Group Code': row['Product Group Code'],
                'Width (cm)': atWidthChild,
                'Size (cm)': atSizeChild,
                'Height (cm)': atHeightChild,
                'EN Description 1': atENDesc1Child,
                'EN Description 2': atENDesc2Child,
                'TR Composition for Export': atTRCompositionChild if (row['Sourcing Method'] == 'Global Supplier') else '',
                'EN Composition for Export': atENCompositionChild if (row['Sourcing Method'] == 'Global Supplier') else '',
                'Supplier Name': row['Supplier'],
                'SKU Supplier Code': row['Supplier Code'],
                'SKU Item Code': row['Item Code'],
                'SKU Color Code': row['Color Code'],
                'Dimension Code': row['Dimension Code'],
                'New Contact': row['Contact'],
                'Product Length (cm)': row['Length (cm)'],
                'Product Width (cm)': row['Width (cm)'],
                'Product Height (cm)': row['Height (cm)'],
                'Handmade': atHandmade,
                'HS Code': '',
                'SC Hub Managed': atSCHubManaged,
                'SC Hub Supplied': atSCHubSupplied,
                'FDA Code': '',
                'Sent to WMS': 'false',
                'Billable': atBillableChild,
                'BOM Type': atBOMType,
                'WMS Code': atWMSCodeChild,
                'ForSlim4': 'true',
                'Planning Item': 'false',
                'Stocked Item': 'true',
                'Logistic Item': 'true',
                'VWH Code': row['VWH Code'],
                'Product Category': row['Product Category'],
                'Sourcing Method': row['Sourcing Method'],
                'Status': row['Status']
            }
            itemList.append(itemDictChild)
            
            BOMComponentDictChild = {
                'Parent Item No.': row['Code'],
                'Line No.': atLineNoChild,
                'Type': 'Item',
                'No.': atCodeChild,
                'Unit of Measure Code': 'ADET',
                'Quantity per': '1'
            }
            BOMComponentList.append(BOMComponentDictChild)
            
dataItem = pd.DataFrame(itemList)
dataBOMComponent = pd.DataFrame(BOMComponentList)
dataItemParentLocation = pd.DataFrame(compatibilityList)

with pd.ExcelWriter('output_new.xlsx') as writer:
    dataItem.to_excel(writer, sheet_name = 'Item')
    dataBOMComponent.to_excel(writer, sheet_name = 'BOM Component')
    dataItemParentLocation.to_excel(writer, sheet_name = 'Item Parent Location')
    
elapsed = time.time() - t
print(elapsed)

1.659916639328003
